<a href="https://colab.research.google.com/github/AaronSanchezBelber/Capstone_Repository/blob/master/Api_Binance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1) OBTENCIÓN DE LOS DATOS. Es necesario obtener datos en tiempo real?**

**2) GENERAR ESTRATEGIAS. Donde podemos implementar los algoritmos de ML? Necesitamos un enfoque**

**3)APLICAR UN BOT DE INVERSIÓN A UNA PLATAFORMA DE TRADING?. Propuesta Metatrader 5**

In [1]:
#Curva de profitability del ACO. 1-2años. Sacar conclusiones. Patrón?. Qué sucede ese primer día?. Qué factor tienen, corr(), entre los tipos de interés y las cryptos?. Liquidaciones. Que pasa con los apacalancamientos(una forma de trading)
#Qué beneficio esperado hay?. Hacer un bot que encuentre esas situaciones. AVE, otra estategía(acalancamiento). ccxt. 1m-ms. 1h-1m 1D-1m. Patrones definidos? sacar las ico de binance, ej:100primeras. Perfomance contra el bitcoin(comparar el bitcoin con la ico que salga)
#Mirar el retorno(F()) de la inversión para cada ICO 1,5,1h,1d,1m,3m,5m. Calcular el retorno del bitcoin. Comparalos. 
#Invertir en ICOs, si o no?. Si invierto en ICOs cuando vendo?
#Nivel rentabilidad riesgo. 
#CLustering, rentabiliad. precio. volumen total de precio. 
#No es necesario el tiempo real
#Empezar de manera sencilla y luego sofisticar la estrategia
#Buscar estudio de rentabilidades IPO
#https://www.epochconverter.com/. (1667527140000 esto es una fecha)

**PROYECTO CAPSTONE: CRYPTO BOT DE TRADING**





0) OBTENCIÓN DE DATOS: USAREMOS LA API DE BINANCE PARA OBTENER LOS DATOS


In [2]:
#Comenzamos instalando python/binance
!pip install python-binance mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 65 kB 798 kB/s 
     |████████████████████████████████| 71 kB 3.5 MB/s 
     |████████████████████████████████| 45 kB 979 kB/s 
     |████████████████████████████████| 292 kB 46.7 MB/s 
     |████████████████████████████████| 106 kB 18.0 MB/s 
     |████████████████████████████████| 749 kB 37.2 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2022.6.2
    Uninstalling regex-2022.6.2:
      Successfully uninstalled regex-2022.6.2


In [3]:
#importamos las librerias necesarias
import pandas as pd                           #esta libreria nos servira para manipular y visualizar datos
import os                                     #nos permite crear una variable una variable oculta

from binance import Client                    #Software que recoge informacion de un servidor (En nuestro caso necesita unas credenciales)
from binance import ThreadedWebsocketManager ##Refresca necesita el package sleep
from time import sleep                        #Se utiliza con ThreadedWeb... pero no sabemos darle uso
from binance import BinanceSocketManager     ##Refresca
from binance import ThreadedDepthCacheManager ##???

In [4]:
#Guardamos nuestras credenciales como variables ocultas mediante os
os.environ['binance_api'] = 'OI0aiCyAYktPpILpNh5nJSVmtygqNnXMXcBVjSrI9lx8MkMv1c5WwruIBhGw5jia'      #Utilizamos estas credentials tipo test
os.environ['binance_secret'] =  'A7AAsjWJ9tt8C1HBjISWbFdynvNgwLNHoSnJaUog1C7bYngaAr1tDRyG2jPXuWFP'  #Cuidado manten secretas

In [5]:
#Ahora asignamos nuestra variable oculta a una variable
apikey = os.environ['binance_api'] 
secret = os.environ['binance_secret']

In [6]:
#Utilizo las claves de Ivar, ya que tengo restricciones xon las anteriores
ivar_api = 'emZw0cRFRM8RfXtJ8iyVw5b5h5b1MOfmqzcIhtPTTbYumx3WxSvEoJLUM8wMMIhV'
ivar_secret = 'uGwleBRfoxE8kz3Q4KNNYIm8UxM7zfLWYwqDee8R862I8RnbkUKBSpV7Q1D0SiXU'

In [7]:
#Añadimos la funcion clientes junto a sus credenciales, para poder acceder a API
client = Client(ivar_api, ivar_secret)

0.1) OBTENCIÓN DE DATOS, MANIPULACIÓN Y VISUALIZACIÓN DE DATOS

In [8]:
#Todas las criptos disponibles en Binance, en DataFrame
tickers = client.get_all_tickers()
tickers_df = pd.DataFrame(tickers)
tickers_df

,symbol,price
0,ETHBTC,0.07700300
1,LTCBTC,0.00329800
2,BNBBTC,0.01663200
3,NEOBTC,0.00042700
4,QTUMETH,0.00184400
...,...,...
2118,APTBRL,38.21000000
2119,QKCBUSD,0.01156000
2120,OSMOBTC,0.00007695
2121,OSMOUSDT,1.63400000


0.1) OBTENCIÓN DE DATOS: USAMOS WEBSOCKET PARA OBTENER DATOS EN TIEMPO REAL

In [9]:
#https://pypi.org/project/websocket-client/ #Para saber usar websocket
#https://github.com/binance/binance-spot-api-docs/blob/master/web-socket-streams.md #Datos en tiempo real disponibles

In [10]:
!pip install websocket
!pip install websocket-client
import websocket, json


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 195 kB 14.4 MB/s 
     |████████████████████████████████| 6.0 MB 66.4 MB/s 
     |████████████████████████████████| 511 kB 59.0 MB/s 
     |████████████████████████████████| 254 kB 44.5 MB/s 
  Created wheel for websocket: filename=websocket-0.2.1-py3-none-any.whl size=192135 sha256=620d8af9ea7b38574f6f19159fdf73db102d2e2aee39ced3956006026cb607fb
  Stored in directory: /root/.cache/pip/wheels/e2/9b/2b/6324f465ec5543e33a31c2692c01c1760173bbb98b37dbaee8
Successfully built websocket
  Attempting uninstall: greenlet
    Found existing installation: greenlet 1.1.3.post0
    Uninstalling greenlet-1.1.3.post0:
      Successfully uninstalled greenlet-1.1.3.post0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.7 MB/s 


In [11]:
activo_btcusd = 'btcusd'
interval_1m = '1m'

In [12]:
#La url/socket para que funcione. Le estamos diciendo que nos pase velas japonesas cada segundo de bitcoin
socket = f'wss://stream.binance.com:9443/ws/{activo_btcusd}/@kline_{interval_1m}'

In [13]:
socket

'wss://stream.binance.com:9443/ws/btcusd/@kline_1m'

In [14]:
#making candle sticks
def on_message(ws, message):
  json.message = json.loads(message)
  candle = json_message['k']
  is_candle_closed = candle['x']
  close = candle['c']
  high = candle['h']
  low = candle['l']
  vol = candle['v']

  print(close)
  print(high)
  print(low)
  print(vol)

def on_close(ws):
  print('Connection Closed')



In [15]:
#usamos ws para generar una funcion diciendole que queremos, para esto necesitamos que mensaje y que cierre queremos haciendo dos funciones previas
ws = websocket.WebSocketApp(socket,on_message = on_message, on_close = on_close)

In [ ]:
ws.run_forever()

In [ ]:
import websocket
import _thread
import time

def on_message(ws, message):
    print(message)

def on_error(ws, error):
    print(error)

def on_close(ws, close_status_code, close_msg):
    print("### closed ###")

def on_open(ws):
    print("Opened connection")

if __name__ == "__main__":
    websocket.enableTrace(True)
    ws = websocket.WebSocketApp(socket,
                              on_open=on_open,
                              on_message=on_message,
                              on_error=on_error,
                              on_close=on_close)

  